In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import copy

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]

def equals(a, b, tol = 1E-3):
    if abs(a-b) <= tol:
        return True
    else:
        return False
    
def equals_array(arr_1, arr_2, tol = 1E-2):
    if np.linalg.norm(arr_1 - arr_2) <= tol:
        return True
    else:
        return False
    

In [3]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

In [4]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# np.linalg.norm(arr_1)

# Download Groups, Routes to Edges Data:

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
# df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_3.csv')

# df_od_flow_data
# df_data

In [6]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [7]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.5396169,
  'Demand (eligible group, 1)': 60.02888245620001,
  'VoT (eligible group, 1)': 0.04180157639979069,
  'Demand (eligible group, 2)': 36.752377014000004,
  'VoT (eligible group, 2)': 0.1494057158119658,
  'Demand (ineligible group, 1)': 93.71856138570001,
  'VoT (ineligible group, 1)': 0.29949880593262945,
  'Demand (ineligible group, 2)': 134.75871571800002,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108032610004,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986,
  'Demand (eligible group, 1)': 9.607422146280001,
  'VoT (eligible group, 1)': 0.04180157639979069,
  'Demand (eligible group, 2)': 5.8820951916,
  'VoT (eligible group, 2)': 0.149

In [8]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [9]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

edge_to_od_dict = {}
num_edges = int(np.max(od_to_edges_array)) + 1
# print("num_edges:", num_edges)

for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
# od_to_edges_array
# edge_to_od_dict

In [10]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 60.02888246  36.75237701  93.71856139 134.75871572 287.28108033]
 [  9.60742215   5.88209519  14.99934274  21.56768237  45.97837741]
 [111.52479072  68.28048412 174.11523449 250.36177509 533.72578417]
 [ 25.07588823  15.35258463  39.14909081  56.29281031 120.00603653]
 [ 95.06622324  58.20381015 148.41971587 213.41397053 454.95978264]
 [ 23.76644387  14.550884    37.10475421  53.35324134 113.73940995]
 [150.04450532  91.86398285 234.25315626 336.83460377 718.07013259]
 [ 25.30172796  35.37078297  62.99613901  74.8724603   59.63978735]
 [  5.68890543   7.95285759  14.16421351  16.83451606  13.40956279]
 [ 21.56665392  30.14930191  53.6965669   63.81969017  50.83568424]
 [  5.39189555   7.5376499   13.42471953  15.95560927  12.70946808]
 [ 34.04016665  47.58676358  84.75306798 100.73110539  80.23753567]
 [ 10.40251464  13.91885761  23.5888008   37.36114412  61.2429735 ]
 [ 39.43629223  52.76686989  89.42595844 141.63738759 232.1742275 ]
 [  9.85945787  13.19223236  22.35736222  35.410

In [11]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [12]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [13]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

In [14]:
# num_edges = 7
num_gp_lanes = 3

num_el = 3
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]
    

In [15]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_list = [lambda_E, lambda_R, lambda_I]

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# directory_inits = '../data/opt_values___2_el_groups/'
directory_inits = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
df_inits = pd.read_csv(directory_inits + filename_segment + '___tau_B_stats_CBCP.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)


filename_segment: 1_1_1

argmin_tau:
 [[0.92 0.28 0.46 0.48 0.93]
 [0.95 0.32 0.79 0.58 0.71]
 [0.52 2.24 2.07 1.35 2.2 ]
 [1.26 0.86 1.71 1.38 2.46]
 [2.13 2.29 2.26 0.59 1.31]
 [0.53 0.44 0.69 0.51 0.38]
 [0.78 0.26 0.48 0.36 0.74]]

argmin_B:
 7.02


##  <font color='blue'>Import argmin_y directly (requires storing it from the other Jupyter notebook file).</font> 

In [16]:
# ## Set lambdas:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# directory_path = '../data/opt_values___2_el_groups/'
directory_path = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

y_CBCP_opt_array = pd.read_csv(directory_path + filename_segment + '___y_CBCP.csv').to_numpy()

y_CBCP_opt = {}
for row_index in range(y_CBCP_opt_array.shape[0]):
    key = tuple(y_CBCP_opt_array[row_index,:5].astype(int))
    y_CBCP_opt[key] = y_CBCP_opt_array[row_index,-1]

# y_CBCP_array
y_CBCP_opt


{(0, 0, 0, 0, 0): 60.02869961796074,
 (0, 0, 0, 1, 0): 0.0001819574770083,
 (0, 1, 0, 0, 0): 36.75219523807682,
 (0, 1, 0, 1, 0): 0.0001812366809695,
 (0, 2, 0, 0, 0): 93.71837697431876,
 (0, 2, 0, 1, 0): 0.0001830363136289,
 (0, 3, 0, 0, 0): 6.964950318542129e-05,
 (0, 3, 0, 1, 0): 134.75864606849683,
 (0, 4, 0, 0, 0): 0.0007816896292494,
 (0, 4, 0, 1, 0): 287.2802986364708,
 (0, 0, 0, 0, 1): 60.02831162115661,
 (0, 0, 0, 1, 1): 0.0005699542811374,
 (0, 1, 0, 0, 1): 36.75181054075323,
 (0, 1, 0, 1, 1): 0.0005659340045545,
 (0, 2, 0, 0, 1): 93.71798713877182,
 (0, 2, 0, 1, 1): 0.0005728718605457,
 (0, 3, 0, 0, 1): 0.0001994346430738,
 (0, 3, 0, 1, 1): 134.75851628335695,
 (0, 4, 0, 0, 1): 0.002908445758797,
 (0, 4, 0, 1, 1): 287.27817188034123,
 (0, 0, 0, 0, 2): 60.02853284067693,
 (0, 0, 0, 1, 2): 0.0003487347608159,
 (0, 1, 0, 0, 2): 36.752029656586544,
 (0, 1, 0, 1, 2): 0.0003468181712556,
 (0, 2, 0, 0, 2): 93.71820937823016,
 (0, 2, 0, 1, 2): 0.0003506324022056,
 (0, 3, 0, 0, 2): 0

# Compute Aggregated Flows, Travel Times, and Cost Metrics

In [17]:
def compute_y_agg_x(y, edge_to_od_dict, num_edges, T):
    
    y_agg = {}
    x = {}
    
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                y_agg[(e, k, t, "in")] = sum(y[(od, g, e, k, t)] \
                                                for od in edge_to_od_dict[e] for g in in_indices)
                y_agg[(e, k, t, "el")] = sum(y[(od, g, e, k, t)] \
                                                for od in edge_to_od_dict[e] for g in el_indices)

                x[(e, k, t)] = sum(y[(od, g, e, k, t)] \
                                        for od in edge_to_od_dict[e] for g in in_indices + el_indices)

    return y_agg, x


def compute_travel_times(y_agg, x, edge_to_od_dict, coeff_input, num_gp_lanes, num_edges, T):
    
    travel_times = {}
    avg_travel_times = {}
    
    for e in range(num_edges):
        for t in range(T):
            travel_times[(e, 0, t)] = coeff_input[0, e] \
                + coeff_input[1, e] * max(x[(e, 0, t)] - coeff_input[2, e], 0.0)
            travel_times[(e, 1, t)] = coeff_input[0, e] \
                + coeff_input[1, e] * max(x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)
    
    for e in range(num_edges):
        avg_travel_times[(e, 'ex')] = sum([travel_times[(e, 0, t)] for t in range(T)]) / T
        avg_travel_times[(e, 'gp')] = sum([travel_times[(e, 1, t)] for t in range(T)]) / T
        
#         print()
#         print("e:", e)
#         print("sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)):", \
#              sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)))
#         print("sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)):", \
#              sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)))
        
        avg_travel_times[(e, 'el')] = sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T))
        avg_travel_times[(e, 'in')] = sum(y_agg[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
    
    avg_travel_times['ex'] = sum([travel_times[(e, 0, t)] for e in range(num_edges) for t in range(T)]) / T
    avg_travel_times['gp'] = sum([travel_times[(e, 1, t)] for e in range(num_edges) for t in range(T)]) / T

    avg_travel_times['el'] = sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for e in range(num_edges) for k in range(2) for t in range(T)) \
                                / sum(y_agg[(e, k, t, 'el')] for e in range(num_edges) for k in range(2) for t in range(T))
    avg_travel_times['in'] = sum(y_agg[(e, k, t, 'in')] * travel_times[(e, k, t)] for e in range(num_edges) for k in range(2) for t in range(T)) \
                                / sum(y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    
    return travel_times, avg_travel_times

def compute_percent_on_express(y_agg, x, num_edges, T):
    
    percent_on_express = {}
    
    for e in range(num_edges):
        percent_on_express[(e, 'el')] = sum(y_agg[(e, 0, t, 'el')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T))
        percent_on_express[(e, 'in')] = sum(y_agg[(e, 0, t, 'in')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
        percent_on_express[(e, 'all')] = sum(y_agg[(e, 0, t, 'el')] + y_agg[(e, 0, t, 'in')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] + y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
    
    percent_on_express['el'] = sum(y_agg[(e, 0, t, 'el')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'el')] for e in range(num_edges) for k in range(2) for t in range(T))
    percent_on_express['in'] = sum(y_agg[(e, 0, t, 'in')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    percent_on_express['all'] = sum(y_agg[(e, 0, t, 'el')] + y_agg[(e, 0, t, 'in')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'el')] + y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    
    return percent_on_express

# def compute_obj_CBCP_together(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T):
#     obj = {}
    
#     lambda_E, lambda_R, lambda_I = lambda_list
    
#     for e in range(num_edges):
#         obj[(e, 'E')] = sum( y[(od, g, e, k, t)] * VoT_array[od, g, t] * travel_times[(e, k, t)] \
#                             for od in edge_to_od_dict[e] for g in el_indices for k in range(2) for t in range(T) )

#         obj[(e, 'R')] = sum( y[(od, g, e, 0, t)] * tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[(e, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
#     obj['total'] = sum([obj[e] for e in range(num_edges)])
    
#     return obj


def compute_obj_CBCP(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    for e in range(num_edges):
        for t in range(T):
            obj[(e, t, 'E')] = sum( y[(od, g, e, k, t)] * VoT_array[od, g, t] * travel_times[(e, k, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices for k in range(2))

            obj[(e, t, 'R')] = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
            
            obj[(e, t, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
                                   for od in edge_to_od_dict[e] for g in in_indices) \
                                + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                      for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t)] = lambda_E * obj[(e, t, 'E')] - lambda_R * obj[(e, t, 'R')] + lambda_I * obj[(e, t, 'I')]  
        
        obj[(e, 'E')] = sum([obj[(e, t, 'E')] for t in range(T)])
        obj[(e, 'I')] = sum([obj[(e, t, 'I')] for t in range(T)])
        obj[(e, 'R')] = sum([obj[(e, t, 'R')] for t in range(T)])
        
        obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
    obj['E'] = sum([obj[(e, 'E')] for e in range(num_edges)])
    obj['I'] = sum([obj[(e, 'I')] for e in range(num_edges)])
    obj['R'] = sum([obj[(e, 'R')] for e in range(num_edges)])
    
    obj['total'] = sum([obj[e] for e in range(num_edges)])
    
    return obj


In [18]:
y_agg_CBCP_opt, x_CBCP_opt \
    = compute_y_agg_x(y = y_CBCP_opt, \
                      edge_to_od_dict = edge_to_od_dict,\
                      num_edges = num_edges, \
                      T = T)

travel_times_CBCP_opt, avg_travel_times_CBCP_opt \
    = compute_travel_times(y_agg = y_agg_CBCP_opt, \
                           x = x_CBCP_opt, \
                           edge_to_od_dict = edge_to_od_dict,\
                           coeff_input = coeff_input, \
                           num_gp_lanes = num_gp_lanes,\
                           num_edges = num_edges, \
                           T = T)

percent_on_express_CBCP_opt \
    = compute_percent_on_express(y_agg = y_agg_CBCP_opt,\
                                 x = x_CBCP_opt, \
                                 num_edges = num_edges, \
                                 T = T)

lambda_list = [lambda_E, lambda_R, lambda_I]

obj_CBCP_opt = compute_obj_CBCP(y = y_CBCP_opt, \
                       x = x_CBCP_opt, \
                       travel_times = travel_times_CBCP_opt, \
                       VoT_array = VoT_array, \
                       tau = argmin_tau, \
                       edge_to_od_dict = edge_to_od_dict, \
                       lambda_list = lambda_list, \
                       num_edges = num_edges, \
                       T = T)

In [19]:
# y_agg_CBCP_opt
# x_CBCP_opt

# travel_times_CBCP_opt
# avg_travel_times_CBCP_opt
# percent_on_express_CBCP
obj_CBCP_opt


{(0, 0, 'E'): 494.20111173136866,
 (0, 0, 'R'): 356.1766286682868,
 (0, 0, 'I'): 8491.531739181053,
 (0, 0): 8629.556222244135,
 (0, 1, 'E'): 453.62391389674644,
 (0, 1, 'R'): 149.44405133712883,
 (0, 1, 'I'): 7635.538210349411,
 (0, 1): 7939.718072909029,
 (0, 2, 'E'): 456.19559092457297,
 (0, 2, 'R'): 318.97318547870435,
 (0, 2, 'I'): 7804.9433616618,
 (0, 2): 7942.165767107668,
 (0, 3, 'E'): 432.21477710138146,
 (0, 3, 'R'): 367.96071801226174,
 (0, 3, 'I'): 7986.8558489429,
 (0, 3): 8051.109908032019,
 (0, 4, 'E'): 506.0717659553741,
 (0, 4, 'R'): 281.14181216222755,
 (0, 4, 'I'): 8868.924589263013,
 (0, 4): 9093.85454305616,
 (0, 'E'): 2342.3071596094437,
 (0, 'I'): 40787.79374939818,
 (0, 'R'): 1473.6963956586094,
 0: 41656.404513349014,
 (1, 0, 'E'): 925.9586181836324,
 (1, 0, 'R'): 0.009078186900784026,
 (1, 0, 'I'): 12324.309096391165,
 (1, 0): 13250.258636387896,
 (1, 1, 'E'): 789.0235617045297,
 (1, 1, 'R'): 170.7938826932169,
 (1, 1, 'I'): 10990.671455722942,
 (1, 1): 11608

## Store into opt data array:

In [20]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# argmin_B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express_CBCP_opt[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express_CBCP_opt[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express_CBCP_opt[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_times_CBCP_opt[(e, 'ex')] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_times_CBCP_opt[(e, 'gp')] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_CBCP_opt[(e, 'E')] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_CBCP_opt[(e, 'I')] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_CBCP_opt[(e, 'R')] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj_CBCP_opt[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [21]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save

,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.92,0.28,0.46,0.48,0.93,0.61,7.02,17.32,20.10,16.07,1.40,1.70,2342.31,40787.79,1473.70,41656.40
e=2,0.95,0.32,0.79,0.58,0.71,0.67,0.00,9.66,7.73,10.66,2.22,2.54,4258.28,59346.60,1004.31,62600.57
e=3,0.52,2.24,2.07,1.35,2.20,1.68,0.00,19.21,26.27,15.60,5.56,6.38,13499.97,194351.15,5538.88,202312.24
e=4,1.26,0.86,1.71,1.38,2.46,1.53,0.00,17.11,32.81,9.41,1.23,1.98,3309.58,53458.26,2700.16,54067.69
e=5,2.13,2.29,2.26,0.59,1.31,1.72,0.00,21.31,52.47,5.84,6.96,7.80,18305.56,270230.44,2210.30,286325.71
e=6,0.53,0.44,0.69,0.51,0.38,0.51,0.00,15.85,33.98,6.95,1.57,1.82,3642.64,53182.58,825.22,56000.00
e=7,0.78,0.26,0.48,0.36,0.74,0.52,0.00,16.70,43.79,3.41,2.49,2.75,5972.81,85521.14,368.82,91125.13


## Initialize DBCP flows

In [22]:
# Function for transforming arrays of size (n, ) into arrays of size (n, 1)

def pad_dim(arr_or_list):
    arr = np.array(arr_or_list)
    assert len(arr.shape) == 1, "We must have len(arr.shape) == 1 to proceed"
    arr_len = arr.shape[0]
    return arr.reshape((arr_len, 1))

# Function for filling a vector from the bottom up to some value:

def fill_from_bottom(arr_or_list, val):
    arr = np.array(arr_or_list)
    assert np.all(arr >= -1E-3), "We must have all entries of arr >= 0.0."
    assert val >= -1E-3, "We must have val >= 0.0."
    
    arr_fill_from_bottom = np.zeros(arr.shape)
    
    index_boundary = max([index for index in range(arr.shape[0]) if np.sum(arr[index:]) >= val - 1E-3])
    
    for index in range(arr.shape[0]):
        if index > index_boundary:
            arr_fill_from_bottom[index] = arr[index]
        elif index < index_boundary:
            arr_fill_from_bottom[index] = 0.0
        else:
            arr_fill_from_bottom[index] = arr[index_boundary] - (np.sum(arr[index_boundary:]) - val)
            
        if abs(arr_fill_from_bottom[index]) <= 1E-3:
            arr_fill_from_bottom[index] = 0.0
            
#         print("index:", index)
#         print("index_boundary:", index_boundary)
#         print("arr_fill_from_bottom[index]", arr_fill_from_bottom[index])
            
        assert arr_fill_from_bottom[index] >= 0.0, "We must have arr_fill_from_bottom[index] >= 0.0"

    assert abs(np.sum(arr_fill_from_bottom) - val) <= 1E-3, \
        "We must have np.sum(arr_fill_from_bottom) == val"
    
    return arr_fill_from_bottom, index_boundary


In [23]:
start_time = time.time()

dict_VoTs_demands_annotated = {}

# directory_path = "../data/VoTs_demands_sorted___2_el_groups/"
directory_path = "../data/VoTs_demands_sorted___" + str(num_el) + "_el_groups/"

for e in range(num_edges):
    for t in range(T):
        
        # Ineligible users:
        
        filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
        df_data_in = pd.read_csv(directory_path + filename_in)
        dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()
        
        # Eligible users:
        
        filename_el = str(e) + "_" + str(t) + "_" + "el.csv"
        df_data_el = pd.read_csv(directory_path + filename_el)
        dict_VoTs_demands_annotated[(e, t, "el")] = df_data_el.to_numpy()

end_time = time.time()
print("Time:", end_time - start_time)

Time: 0.21805620193481445


In [24]:
def y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t, y_agg_el_at_e_t, \
                           VoTs_demands_annotated_in, VoTs_demands_annotated_el, \
                           e, t):
    
    y_DBCP_at_e_t = {}
    
    ## Ineligible user flows:

#     VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
    demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
        = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg_in_at_e_t)
    demands_annotated_in_gp_DBCP \
        = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP

    assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
    assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"

    for row_index in range(VoTs_demands_annotated_in.shape[0]):
        od, g = VoTs_demands_annotated_in[row_index, 2:]
        od, g = int(od), int(g)
        y_DBCP_at_e_t[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
        y_DBCP_at_e_t[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]

    ## Eligible user flows:

#     VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
    demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
        = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg_el_at_e_t)
    demands_annotated_el_gp_DBCP \
        = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP

    assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
    assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"

    for row_index in range(VoTs_demands_annotated_el.shape[0]):
        od, g = VoTs_demands_annotated_el[row_index, 2:]
        od, g = int(od), int(g)
        y_DBCP_at_e_t[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
        y_DBCP_at_e_t[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]

    ## Compute alpha:

    VoT_in_boundary = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
    VoT_el_boundary = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
    alpha = 1 - VoT_el_boundary / VoT_in_boundary

    assert 0.0 <= alpha <= 1.0, "We must have 0.0 <= alpha <= 1.0."
    
    return y_DBCP_at_e_t, VoT_in_boundary, VoT_el_boundary, alpha


In [25]:
y_DBCP_init = {}
init_alpha = np.zeros((num_edges, T))
VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        
        y_DBCP_init_at_e_t, VoT_in_boundary[e, t], VoT_el_boundary[e, t], init_alpha[e, t] \
            = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
                                     y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
                                     VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
                                     VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
                                     e = e, \
                                     t = t)
        y_DBCP_init = {**y_DBCP_init, **y_DBCP_init_at_e_t}
        
        assert 0.0 <= init_alpha[e, t] <= 1.0, "We must have 0.0 <= init_alpha[e, t] <= 1.0."
        
        # y_agg_CBCP_opt[(e, 0, t, 'el')]


In [26]:
# demand_array.shape
# VoT_array.shape

In [27]:
def y_boundary_toggle_setup(y, demand_array, VoT_array, el_indices, in_indices, \
                            VoT_el_boundary, VoT_in_boundary, num_edges, T):
        
    y_agg_boundary_to_toggle = {}
    y_agg_el_ex_low = np.zeros((num_edges, T))
    y_agg_el_ex_high = np.zeros((num_edges, T))

    y_agg_el_at_boundary_VoT = np.zeros((num_edges, T))
    y_agg_in_at_boundary_VoT = np.zeros((num_edges, T))
    demand_el_at_boundary_VoT = np.zeros((num_edges, T))
    demand_in_at_boundary_VoT = np.zeros((num_edges, T))
    y_agg_el_above_boundary_VoT = np.zeros((num_edges, T))
    y_agg_in_above_boundary_VoT = np.zeros((num_edges, T))
    y_agg_at_boundary_VoT = np.zeros((num_edges, T))

    for e in range(num_edges):
        for t in range(T):
            print()
            print("e:", e)
            print("t:", t)
            print()

            y_agg_el_at_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
                       if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
            y_agg_in_at_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
                       if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
            y_agg_at_boundary_VoT[e, t] = y_agg_el_at_boundary_VoT[e, t] + y_agg_in_at_boundary_VoT[e, t]

    #         y_agg_at_boundary_VoT[e, t] \
    #             = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
    #                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3]) \
    #                 + sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
    #                        if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])

            demand_el_at_boundary_VoT[e, t] \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
                       if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
            demand_in_at_boundary_VoT[e, t] \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
                       if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])

            if abs(y_agg_el_at_boundary_VoT[e, t] - demand_el_at_boundary_VoT[e, t]) <= 1E-3 and \
                    abs(y_agg_in_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t]) <= 1E-3:
                y_agg_boundary_to_toggle[(e, t)] = False
            else:
                y_agg_boundary_to_toggle[(e, t)] = True

            y_agg_el_above_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
                       if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])

            y_agg_in_above_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
                       if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])

            demand_el_above_boundary_VoT \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
                       if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
            demand_in_above_boundary_VoT \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
                       if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])

            # print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
            # print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
            # print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
            # print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)

            y_agg_el_ex_low[e, t] = demand_el_above_boundary_VoT \
                + max(y_agg_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t], 0.0)

            y_agg_el_ex_high[e, t] = demand_el_above_boundary_VoT \
                + min(y_agg_at_boundary_VoT[e, t], demand_el_at_boundary_VoT[e, t])
            
#             print("e:", e)
#             print("t:", t)
            
            print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
            print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
            
            print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
            print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)

            assert y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t] + 1E-3, \
                "We must have y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t]."

            assert abs(y_agg_el_above_boundary_VoT[e, t] - demand_el_above_boundary_VoT) <= 1E-3, \
                "We should have y_agg_el_above_boundary_VoT[e, t] = demand_el_above_boundary_VoT"

            assert abs(y_agg_in_above_boundary_VoT[e, t] - demand_in_above_boundary_VoT) <= 1E-3, \
                "We should have y_agg_in_above_boundary_VoT[e, t] = demand_in_above_boundary_VoT"
    
    return y_agg_boundary_to_toggle, y_agg_el_ex_low, y_agg_el_ex_high


In [28]:
y_agg_boundary_to_toggle, y_agg_el_ex_low, y_agg_el_ex_high \
    = y_boundary_toggle_setup(y = y_DBCP_init, \
                                 demand_array = demand_array, \
                                 VoT_array = VoT_array, \
                                 el_indices = el_indices, \
                                 in_indices = in_indices, \
                                 VoT_el_boundary = VoT_el_boundary, \
                                 VoT_in_boundary = VoT_in_boundary, \
                                 num_edges = num_edges, \
                                 T = T)

# for e in range(num_edges):
#     for t in range(T):
#         print()
#         print("e:", e)
#         print("t:", t)
#         print()
        
#         assert y_agg_boundary_to_toggle___from_func == y_agg_boundary_to_toggle
#         assert equals(y_agg_el_ex_low___from_func[e, t], y_agg_el_ex_low[e, t])
#         assert equals(y_agg_el_ex_high___from_func[e, t], y_agg_el_ex_high[e, t])


# y_agg_boundary_el_ex_high - y_agg_boundary_el_ex_low



e: 0
t: 0

y_agg_el_above_boundary_VoT[e, t]: 174.11523449427
demand_el_above_boundary_VoT: 174.11523449427
y_agg_in_above_boundary_VoT[e, t]: 0.0
demand_in_above_boundary_VoT: 0

e: 0
t: 1

y_agg_el_above_boundary_VoT[e, t]: 528.495530460408
demand_el_above_boundary_VoT: 528.4955304604081
y_agg_in_above_boundary_VoT[e, t]: 533.72578416871
demand_in_above_boundary_VoT: 533.72578416871

e: 0
t: 2

y_agg_el_above_boundary_VoT[e, t]: 242.13827725687804
demand_el_above_boundary_VoT: 242.13827725687804
y_agg_in_above_boundary_VoT[e, t]: 159.71778736487
demand_in_above_boundary_VoT: 159.71778736487

e: 0
t: 3

y_agg_el_above_boundary_VoT[e, t]: 213.26432530383
demand_el_above_boundary_VoT: 213.26432530383
y_agg_in_above_boundary_VoT[e, t]: 718.0701325858201
demand_in_above_boundary_VoT: 718.0701325858201

e: 0
t: 4

y_agg_el_above_boundary_VoT[e, t]: 93.7185613857
demand_el_above_boundary_VoT: 93.71856138570001
y_agg_in_above_boundary_VoT[e, t]: 120.00603653387998
demand_in_above_boundary_V

## Check if DBCP initialization outperforms CBCP

In [29]:
# def compute_obj_DBCP_together(y, x, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, num_edges, T):
#     obj = {}
    
#     lambda_E, lambda_R, lambda_I = lambda_list
    
#     for e in range(num_edges):
#         obj[(e, 'E')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + (1 - alpha[e, t]) * tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

#         obj[(e, 'R')] = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 0, t)] * tau[e, t] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[(e, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
#     obj['total'] = sum([obj[e] for e in range(num_edges)])
    
#     return obj


def compute_obj_DBCP(y, x, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, num_edges, T):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    for e in range(num_edges):
        for t in range(T):
            
#             print("e:", e)
#             print("t:", t)
            
            obj[(e, t, 'E')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + (1 - alpha[e, t]) * tau[e, t]) \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                for od in edge_to_od_dict[e] for g in el_indices)
            
            obj[(e, t, 'R')] = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] * tau[e, t] \
                                    for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
                                for od in edge_to_od_dict[e] for g in in_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t)] = lambda_E * obj[(e, t, 'E')] - lambda_R * obj[(e, t, 'R')] + lambda_I * obj[(e, t, 'I')]  
        
        obj[(e, 'E')] = sum([obj[(e, t, 'E')] for t in range(T)])
        obj[(e, 'R')] = sum([obj[(e, t, 'R')] for t in range(T)])
        obj[(e, 'I')] = sum([obj[(e, t, 'I')] for t in range(T)])
        
        obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
    obj['E'] = sum([obj[(e, 'E')] for e in range(num_edges)])
    obj['R'] = sum([obj[(e, 'R')] for e in range(num_edges)])
    obj['I'] = sum([obj[(e, 'I')] for e in range(num_edges)])
    
    obj['total'] = sum([obj[e] for e in range(num_edges)])
    
    return obj


def compute_obj_DBCP_at_e_t(y, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, e, t):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[0] + (1 - alpha) * tau) \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[1] \
                                for od in edge_to_od_dict[e] for g in el_indices)
    
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha) * tau \
                        for od in edge_to_od_dict[e] for g in el_indices) \
                        + sum( y[(od, g, e, 0, t)] * tau \
                            for od in edge_to_od_dict[e] for g in in_indices)

    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[0] + tau) \
                        for od in edge_to_od_dict[e] for g in in_indices) \
                    + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[1] \
                        for od in edge_to_od_dict[e] for g in in_indices)

    obj = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

    return obj, obj_E, obj_R, obj_I


In [30]:
def extract_y_at_e_t(y, e, t):
    
    y_at_e_t = {}
    
    for key in y.keys():
        if key[2] == e and key[4] == t:
            y_at_e_t[key] = y[key]
    
    return y_at_e_t

In [31]:
# Compute aggregate flows and other characteristics

y_agg_DBCP_init, x_DBCP_init \
    = compute_y_agg_x(y = y_DBCP_init, \
                      edge_to_od_dict = edge_to_od_dict,\
                      num_edges = num_edges, \
                      T = T)

travel_times_DBCP_init, avg_travel_times_DBCP_init \
    = compute_travel_times(y_agg = y_agg_DBCP_init, \
                           x = x_DBCP_init, \
                           edge_to_od_dict = edge_to_od_dict,\
                           coeff_input = coeff_input, \
                           num_gp_lanes = num_gp_lanes,\
                           num_edges = num_edges, \
                           T = T)

percent_on_express_DBCP_init \
    = compute_percent_on_express(y_agg = y_agg_DBCP_init,\
                                 x = x_DBCP_init, \
                                 num_edges = num_edges, \
                                 T = T)

lambda_list = [lambda_E, lambda_R, lambda_I]

# obj_DBCP_init_together \
#     = compute_obj_DBCP_together(y = y_DBCP_init, \
#                        x = x_DBCP_init, \
#                        travel_times = travel_times_DBCP_init, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        alpha = init_alpha, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

# obj_DBCP_init \
#     = compute_obj_DBCP(y = y_DBCP_init, \
#                        x = x_DBCP_init, \
#                        travel_times = travel_times_DBCP_init, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        alpha = init_alpha, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

# print("travel_times_DBCP_init:", travel_times_DBCP_init)

obj_DBCP_init = {}
for e in range(num_edges):
    for t in range(T):
        
        travel_times_DBCP_init_at_e_t = [travel_times_DBCP_init[(e, k, t)] for k in [0, 1]]

        obj_DBCP_init[(e, t)], obj_DBCP_init[(e, t, 'E')], \
                obj_DBCP_init[(e, t, 'R')], obj_DBCP_init[(e, t, 'I')] \
            = compute_obj_DBCP_at_e_t(y = y_DBCP_init, \
                                      travel_times = travel_times_DBCP_init_at_e_t, \
                                      VoT_array = VoT_array, \
                                      tau = argmin_tau[e, t], \
                                      alpha = init_alpha[e, t], \
                                      edge_to_od_dict = edge_to_od_dict, \
                                      lambda_list = lambda_list, \
                                      e = e, \
                                      t = t)

    obj_DBCP_init[(e, 'E')] = sum([obj_DBCP_init[(e, t, 'E')] for t in range(T)])
    obj_DBCP_init[(e, 'R')] = sum([obj_DBCP_init[(e, t, 'R')] for t in range(T)])
    obj_DBCP_init[(e, 'I')] = sum([obj_DBCP_init[(e, t, 'I')] for t in range(T)])

    obj_DBCP_init[e] = lambda_E * obj_DBCP_init[(e, 'E')] - lambda_R * obj_DBCP_init[(e, 'R')] + lambda_I * obj_DBCP_init[(e, 'I')]  

obj_DBCP_init['total'] = sum([obj_DBCP_init[e] for e in range(num_edges)])


# print("percent_on_express_DBCP_init:\n", percent_on_express_DBCP_init)

# obj_DBCP_init['total']

In [32]:
# print(obj_DBCP_init == obj___from_func)

print("obj_CBCP_opt[total]:", obj_CBCP_opt['total'])
print("obj_DBCP_init[total]:", obj_DBCP_init['total'])
# obj_DBCP_init['total']

obj_CBCP_opt[total]: 794087.7384972068
obj_DBCP_init[total]: 792840.4435672781


In [33]:
## Toggle code

y_DBCP_init_min = {}

obj_DBCP_init_min = np.zeros((num_edges, T))

num_y_toggle = 101

for e in range(num_edges):
    for t in range(T):
        
        print()
        print("e:", e)
        print("t:", t)
        print()
        
        # Compute obj (DBCP) for DBCP init, as baseline:
        
        travel_times_DBCP_init_at_e_t = [travel_times_DBCP_init[(e, k, t)] for k in [0, 1]]
        
        y_DBCP_init_min_at_e_t = extract_y_at_e_t(y_DBCP_init, e, t)
        
        obj_DBCP_init_min_at_e_t, _, _, _ \
            = compute_obj_DBCP_at_e_t(y = y_DBCP_init, \
                                      travel_times = travel_times_DBCP_init_at_e_t, \
                                      VoT_array = VoT_array, \
                                      tau = argmin_tau[e, t], \
                                      alpha = init_alpha[e, t], \
                                      edge_to_od_dict = edge_to_od_dict, \
                                      lambda_list = lambda_list, \
                                      e = e, \
                                      t = t)
        
        if y_agg_boundary_to_toggle[(e, t)]:
            y_agg_el_ex_arr = np.linspace(y_agg_el_ex_low[e, t], y_agg_el_ex_high[e, t], num_y_toggle)
            
            for y_agg_el_ex in y_agg_el_ex_arr:
                
                y_agg_in_ex = x_DBCP_init[(e, 0, t)] - y_agg_el_ex
                
#                 print("y_agg_el_ex:", y_agg_el_ex)
#                 print("y_agg_in_ex:", y_agg_in_ex)
                
                y_DBCP_toggle_at_e_t, _, _, _ \
                    = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_in_ex, \
                                             y_agg_el_at_e_t = y_agg_el_ex, \
                                             VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
                                             VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
                                             e = e, \
                                             t = t)

                obj_toggle_at_e_t, _, _, _ \
                    = compute_obj_DBCP_at_e_t(y = y_DBCP_toggle_at_e_t, \
                                              travel_times = travel_times_DBCP_init_at_e_t, \
                                              VoT_array = VoT_array, \
                                              tau = argmin_tau[e, t], \
                                              alpha = init_alpha[e, t], \
                                              edge_to_od_dict = edge_to_od_dict, \
                                              lambda_list = lambda_list, \
                                              e = e, \
                                              t = t)
                
                if obj_toggle_at_e_t < obj_DBCP_init_min_at_e_t:
                    
                    y_DBCP_init_min_at_e_t = y_DBCP_toggle_at_e_t
                    obj_DBCP_init_min_at_e_t = obj_toggle_at_e_t
        
        y_DBCP_init_min = {**y_DBCP_init_min, **y_DBCP_init_min_at_e_t}
        obj_DBCP_init_min[e, t] = obj_DBCP_init_min_at_e_t


e: 0
t: 0


e: 0
t: 1


e: 0
t: 2


e: 0
t: 3


e: 0
t: 4


e: 1
t: 0


e: 1
t: 1


e: 1
t: 2


e: 1
t: 3


e: 1
t: 4


e: 2
t: 0


e: 2
t: 1


e: 2
t: 2


e: 2
t: 3


e: 2
t: 4


e: 3
t: 0


e: 3
t: 1


e: 3
t: 2


e: 3
t: 3


e: 3
t: 4


e: 4
t: 0


e: 4
t: 1


e: 4
t: 2


e: 4
t: 3


e: 4
t: 4


e: 5
t: 0


e: 5
t: 1


e: 5
t: 2


e: 5
t: 3


e: 5
t: 4


e: 6
t: 0


e: 6
t: 1


e: 6
t: 2


e: 6
t: 3


e: 6
t: 4



In [34]:
# y_DBCP_init_min

print("obj_CBCP_opt[total]:\n", obj_CBCP_opt['total'])
print()
print("obj_DBCP_init[total]:\n", obj_DBCP_init['total'])
print()
print("np.sum(obj_DBCP_init_min):\n", np.sum(obj_DBCP_init_min))
print()

# np.sum(obj_DBCP_init_min)
# obj_CBCP_opt['total'] - np.sum(obj_DBCP_init_min)
print("Improvement fraction:\n", (obj_CBCP_opt['total'] - np.sum(obj_DBCP_init_min)) / obj_CBCP_opt['total'] * 100, "%")

obj_CBCP_opt[total]:
 794087.7384972068

obj_DBCP_init[total]:
 792840.4435672781

np.sum(obj_DBCP_init_min):
 791051.6691466587

Improvement fraction:
 0.38233424385745857 %


In [35]:
# y_DBCP_init_min

# compute_obj_DBCP(y_DBCP_init_min)

# obj_DBCP_init_min = compute_obj_DBCP_at_e_t(y = y_DBCP_init_min, \
#                                               travel_times = travel_times_DBCP_init_at_e_t, \
#                                               VoT_array = VoT_array, \
#                                               tau = argmin_tau[e, t], \
#                                               alpha = init_alpha[e, t], \
#                                               edge_to_od_dict = edge_to_od_dict, \
#                                               lambda_list = lambda_list, \
#                                               e = e, \
#                                               t = t)

y_agg_DBCP_init_min, x_DBCP_init_min = compute_y_agg_x(y = y_DBCP_init_min, \
                                                       edge_to_od_dict = edge_to_od_dict, \
                                                       num_edges = num_edges, \
                                                       T = T)

travel_times_DBCP_init_min, _ = compute_travel_times(y_agg = y_agg_DBCP_init_min, \
                                                     x = x_DBCP_init_min, \
                                                     edge_to_od_dict = edge_to_od_dict, \
                                                     coeff_input = coeff_input, \
                                                     num_gp_lanes = num_gp_lanes, \
                                                     num_edges = num_edges, \
                                                     T = T)

obj_DBCP_init_min = compute_obj_DBCP(y = y_DBCP_init_min, \
                                     x = x_DBCP_init_min, \
                                     travel_times = travel_times_DBCP_init_min, \
                                     VoT_array = VoT_array, \
                                     tau = argmin_tau, \
                                     alpha = init_alpha, \
                                     edge_to_od_dict = edge_to_od_dict, \
                                     lambda_list = lambda_list, \
                                     num_edges = num_edges, \
                                     T = T)

obj_DBCP_init_min

{(0, 0, 'E'): 508.56202805299506,
 (0, 0, 'R'): 424.43773313829394,
 (0, 0, 'I'): 8491.456108758535,
 (0, 0): 8575.580403673237,
 (0, 1, 'E'): 466.8428970055452,
 (0, 1, 'R'): 176.85376899265316,
 (0, 1, 'I'): 7636.175272331708,
 (0, 1): 7926.1644003446,
 (0, 2, 'E'): 466.72659557136666,
 (0, 2, 'R'): 338.95080498335903,
 (0, 2, 'I'): 7804.942075512186,
 (0, 2): 7932.717866100194,
 (0, 3, 'E'): 438.95023978010016,
 (0, 3, 'R'): 385.8845238974787,
 (0, 3, 'I'): 7986.846535971481,
 (0, 3): 8039.912251854103,
 (0, 4, 'E'): 526.1878146876564,
 (0, 4, 'R'): 413.5434310916775,
 (0, 4, 'I'): 8868.427774861444,
 (0, 4): 8981.072158457422,
 (0, 'E'): 2407.2695750976636,
 (0, 'R'): 1739.6702621034624,
 (0, 'I'): 40787.84776743535,
 0: 41455.44708042955,
 (1, 0, 'E'): 928.118355249734,
 (1, 0, 'R'): 29.015496942211126,
 (1, 0, 'I'): 12326.136129123346,
 (1, 0): 13225.238987430868,
 (1, 1, 'E'): 804.2691698433244,
 (1, 1, 'R'): 223.33838660147944,
 (1, 1, 'I'): 10994.296368456886,
 (1, 1): 11575.2

## Computing lower bound to societal cost difference

In [36]:
## Computing lower bound to societal cost difference,
# i.e., the societal cost should be lower by at least this much: 

# obj_CBCP_opt

rev_increase = np.zeros((num_edges, T))
y_el_ex_CBCP_opt = np.zeros((num_edges, T))
rev_increase_total = 0.0

for e in range(num_edges):
    for t in range(T):
        y_el_ex_CBCP_opt[e, t] = sum([y_CBCP_opt[(od, g, e, 0, t)] \
                                      for od in edge_to_od_dict[e] for g in el_indices ])
        
        rev_increase[e, t] = (1 - init_alpha[e, t]) * argmin_tau[e, t] * y_el_ex_CBCP_opt[e, t]
        
        rev_increase_total += rev_increase[e, t]

# y_CBCP_opt

# y_el_ex_CBCP_opt

# rev_increase

# rev_increase_total

In [37]:
# np.sum(argmin_tau)
# argmin_tau
# y_CBCP_opt

obj_CBCP_opt

{(0, 0, 'E'): 494.20111173136866,
 (0, 0, 'R'): 356.1766286682868,
 (0, 0, 'I'): 8491.531739181053,
 (0, 0): 8629.556222244135,
 (0, 1, 'E'): 453.62391389674644,
 (0, 1, 'R'): 149.44405133712883,
 (0, 1, 'I'): 7635.538210349411,
 (0, 1): 7939.718072909029,
 (0, 2, 'E'): 456.19559092457297,
 (0, 2, 'R'): 318.97318547870435,
 (0, 2, 'I'): 7804.9433616618,
 (0, 2): 7942.165767107668,
 (0, 3, 'E'): 432.21477710138146,
 (0, 3, 'R'): 367.96071801226174,
 (0, 3, 'I'): 7986.8558489429,
 (0, 3): 8051.109908032019,
 (0, 4, 'E'): 506.0717659553741,
 (0, 4, 'R'): 281.14181216222755,
 (0, 4, 'I'): 8868.924589263013,
 (0, 4): 9093.85454305616,
 (0, 'E'): 2342.3071596094437,
 (0, 'I'): 40787.79374939818,
 (0, 'R'): 1473.6963956586094,
 0: 41656.404513349014,
 (1, 0, 'E'): 925.9586181836324,
 (1, 0, 'R'): 0.009078186900784026,
 (1, 0, 'I'): 12324.309096391165,
 (1, 0): 13250.258636387896,
 (1, 1, 'E'): 789.0235617045297,
 (1, 1, 'R'): 170.7938826932169,
 (1, 1, 'I'): 10990.671455722942,
 (1, 1): 11608

# DBCP Optimization

In [38]:
def proj_tau_alpha(T, num_edges, tau, alpha, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



def proj_tau_alpha_at_e_t(T, num_edges, tau, alpha, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

#     assert tau == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
    print("np.round(np.maximum(tau_feas.value, 0.0), decimals=3):", \
          np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print("np.round(np.maximum(alpha_feas, 0.0), decimals=3):", \
         np.round(np.maximum(alpha_feas, 0.0), decimals=3))
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



In [39]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_DBCP_at_e_t(num_gp_lanes, tau, alpha, od_to_edges_list_full, \
                      demand_array, VoT_array, el_indices, in_indices, \
                      coeff_input, e, t):
    
    arr_indicator = np.zeros((num_edges, T))
    arr_indicator[e, t] = 1.0
    
#     print("type(tau):", type(tau))
#     print("type(alpha):", type(alpha))
#     print("tau.shape:", tau.shape)
#     print("alpha:", alpha.shape)
    
    if type(tau) == float or type(tau) == int or tau.shape == ():
#         print("Modifying tau shape")
        tau = tau * arr_indicator
    if type(alpha) == float or type(alpha) == int or alpha.shape == ():
#         print("Modifying alpha shape")
        alpha = alpha * arr_indicator
    
#     print("arr_indicator:\n", arr_indicator)
#     print("tau:", tau)
#     print("alpha:", alpha)
    
    assert tau.shape == alpha.shape, "We must have tau.shape == alpha.shape!"
    assert np.all((alpha >= -1E-3) & (alpha <= 1.001))
    
    alpha = np.clip(alpha, 0.0, 1.0)
    
#     print("tau (in solve_DBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
#     print("coeff:\n", coeff)
#     print()
    
#     assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
#     assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
#                                             demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, g, e, k, t) = (od, income group, edge, "lane", time)
    
#     od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
#                           for od in range(od_to_edges_array.shape[0])]
    
#     edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#                    for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for g in el_indices:
#             for k in [0, 1, 2]:
            for k in [0, 1]:
                y[(od, g, e, k, t)] = cp.Variable(1)
        for g in in_indices:
            for k in [0, 1]:
                y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for k in [0, 1]:
        x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    func += coeff[0, e] * x[(e, 0, t)] \
            + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[(e, 0, t)] - coeff[2, e], 0))
    func += coeff[0, e] * x[(e, 1, t)] \
            + 0.5 * coeff[1, e] * num_gp_lanes * cp.square(cp.maximum(x[(e, 1, t)]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)

    
    for od in range(num_od):
        for g in el_indices:
            
#             print("alpha[e, t]:", alpha[e, t])
#             print("tau[e, t]:", tau[e, t])
#             print("VoT_array[od, g, t]:", VoT_array[od, g, t])
            
            func += (1 - alpha[e, t]) * tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]
#             func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
        for g in in_indices:
            func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
#     constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) for g in el_indices for k in [0, 1, 2]]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) for g in in_indices + el_indices for k in [0, 1]]
    constraints += [x[(e, k, t)] >= 0.0 for k in [0, 1]]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("alpha:", alpha)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []
    
    ## Edge contraints:
    constraints += [sum( y[(od, g, e, k, t)] for od in edge_to_ods[e] for g in el_indices + in_indices) \
                        == x[(e, k, t)] for k in [0, 1] ]

    ## Group flow constraints:
    for od in edge_to_ods[e]:
        for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
            assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
            od_g_e_t_list += [(od, g, e, t)]

#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]

#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]

#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

#     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
#                     for od in edge_to_ods[e] for g in el_indices]
    constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                    for od in edge_to_ods[e] for g in el_indices]
    constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                    for od in edge_to_ods[e] for g in in_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for od in edge_to_ods[e]:
        for g in el_indices:
#             for k in [0, 1, 2]:
            for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
        for g in in_indices:
            for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


In [40]:
def welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
                       demand_array, VoT_array, num_el, od_to_edges_array, y, \
                       coeff_input, e, t):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    assert np.all((alpha >= -1E-3) & (alpha <= 1.001)), "We must have alpha in [0, 1] component-wise!"
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

#     assert len(tau.shape) == 2, "tau should be 2-dimensional."
#     assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
#     assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros(2)
    x[0] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices + in_indices)
    x[1] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indices + in_indices)

        
#     print()
#     print("in_indices:", in_indices)
#     print("el_indices:", el_indices)
#     print()
    
    ## Compute lane latencies:
    
    ell = np.zeros(2)
    ell[0] = coeff[0, e] + coeff[1, e] * max(x[0] - coeff[2, e], 0)
    ell[1] = coeff[0, e] + coeff[1, e] * max(x[1]/num_gp_lanes - coeff[2, e], 0)    

    # Computing obj_E, obj_I, obj_R, and welfare at edge e and time t:
    
    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[0] + (1 - alpha) * tau ) \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[1] \
                  for od in edge_to_od_dict[e] for g in el_indices)
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[0] + tau) \
                for od in edge_to_od_dict[e] for g in in_indices) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[1] \
                for od in edge_to_od_dict[e] for g in in_indices)
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha) * tau \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 0, t)] * tau \
                for od in edge_to_od_dict[e] for g in in_indices)

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("In the welfare_obj_at_e_t function, -1:")
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Chinmay's Zeroth-Order Algorithm, for DBCP optimization:

## Setup

In [41]:
T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3

# num_el = 3
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_DBCP_direct(T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)

# print("tau_upper_limit:\n", tau_upper_limit)

d = 2

# fraction_tau_max = 0.5

# tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)
# print()
# print("tau[:, :, 0]:", tau[:, :, 0])
# print()
# print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))

# od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
#                          for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# alpha[:, :, 0] = alpha_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

In [42]:
print("argmin_tau:\n", argmin_tau)
print()
print("init_alpha:\n", init_alpha)

argmin_tau:
 [[0.92 0.28 0.46 0.48 0.93]
 [0.95 0.32 0.79 0.58 0.71]
 [0.52 2.24 2.07 1.35 2.2 ]
 [1.26 0.86 1.71 1.38 2.46]
 [2.13 2.29 2.26 0.59 1.31]
 [0.53 0.44 0.69 0.51 0.38]
 [0.78 0.26 0.48 0.36 0.74]]

init_alpha:
 [[0.84338257 0.85084862 0.85706593 0.86353093 0.84168641]
 [0.83200002 0.84995156 0.83773352 0.84321766 0.83352961]
 [0.85407664 0.86165158 0.85706593 0.87755727 0.85395944]
 [0.85393105 0.92504674 0.85680169 0.87571159 0.8467413 ]
 [0.86209681 0.92352439 0.9169529  0.87957208 0.9254939 ]
 [0.84338257 0.92125546 0.84409076 0.87397343 0.92228554]
 [0.86032559 0.91955794 0.91636279 0.88337078 0.86165457]]


## Running zeroth-order gradient descent

In [43]:
time_1 = time.time()


## Set lambdas:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Check lambda:
print("lambda_E:", lambda_E)
print("lambda_R:", lambda_R)
print("lambda_I:", lambda_I)
print()

## Initialize tau, alpha values:

# filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))
# # filename_segment = '4654283787'

# directory_inits = '../data/opt_tolls_subsidies_metrics/'
# df_inits = pd.read_csv(directory_inits + 'inits___' + filename_segment + '.csv')

# print("filename_segment:", filename_segment)

welfare_min_array = np.zeros((num_edges, T))
obj_E_min_at_e_t = np.zeros((num_edges, T))
obj_R_min_at_e_t = np.zeros((num_edges, T))
obj_I_min_at_e_t = np.zeros((num_edges, T))

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)

eta_bar = 5E-4
delta_bar = 5E-4

argmin_tau_new = np.zeros((num_edges, T))
argmin_alpha_new = np.zeros((num_edges, T))

y_opt_DBCP_new = copy.deepcopy(y_DBCP_init_min)

for e in range(num_edges):
    for t in range(T):
        
        print("e:", e)
        print("t:", t)

# for e in [0]:
#     for t in [0]:    
        
        tau_e_t = np.zeros(num_iters)
        tau_perturbed_e_t = np.zeros(num_iters)

        alpha_e_t = np.zeros(num_iters)
        alpha_perturbed_e_t = np.zeros(num_iters)
        
        tau_e_t[0] = argmin_tau[e, t]
        alpha_e_t[0] = init_alpha[e, t]
        
        welfare_min_array[e, t] = obj_DBCP_init_min[(e, t)]
        obj_E_min_at_e_t[e, t] = obj_DBCP_init_min[(e, t, 'E')]
        obj_R_min_at_e_t[e, t] = obj_DBCP_init_min[(e, t, 'R')]
        obj_I_min_at_e_t[e, t] = obj_DBCP_init_min[(e, t, 'I')]
        argmin_tau_new[e, t] = argmin_tau[e, t]
        argmin_alpha_new[e, t] = init_alpha[e, t]
        
        for i in range(num_iters-1):
        
            time_1_at_e_t = time.time()

            print()
            print("Iter:", i)

            eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
            delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
            w_i_unnormalized = np.random.randn(2)
            w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
            
        #     print("w_i:", w_i)
            tau_perturbed_e_t[i] = tau_e_t[i] + delta[i] * w_i[0]
            alpha_perturbed_e_t[i] = alpha_e_t[i] + delta[i] * w_i[1]

        #     if tau_perturbed[e, t, i] < alpha_perturbed[e, t, i] or tau_perturbed[e, t, i] < 0 alpha_perturbed[e, t, i] < 0:

            print("tau_e_t[i], before projection:\n", tau_e_t[i])
            print("alpha_e_t[i], before projection:\n", alpha_e_t[i])
            print("tau_perturbed_e_t[i], before projection:\n", tau_perturbed_e_t[i])
            print("alpha_perturbed_e_t[i], before projection:\n", alpha_perturbed_e_t[i])
            
            print("tau_perturbed_e_t[i]:", tau_perturbed_e_t[i])
            print("alpha_perturbed_e_t[i]:", alpha_perturbed_e_t[i])
#             print("tau_upper_limit:", tau_upper_limit)

            tau_perturbed_e_t[i], alpha_perturbed_e_t[i] \
                = proj_tau_alpha_at_e_t(T, num_edges, tau_perturbed_e_t[i], alpha_perturbed_e_t[i], \
                                        tau_max = tau_upper_limit[e, t])

            print("tau[e, t, i], after projection:\n", tau_e_t[i])
            print("alpha[e, t, i], after projection:\n", alpha_e_t[i])
            print("tau_perturbed[e, t, i], after projection:\n", tau_perturbed_e_t[i])
            print("alpha_perturbed[e, t, i], after projection:\n", alpha_perturbed_e_t[i])
            
#             def solve_DBCP_at_e_t(num_gp_lanes, tau, alpha, od_to_edges_list_full, \
#                       demand_array, VoT_array, el_indices, in_indices, \
#                       coeff_input, e, t):

            print()
#             print("demand_array_temp:", demand_array_temp)
            print("VoT_array.shape:", VoT_array.shape)
            print()

            y_values_at_e_t = \
                solve_DBCP_at_e_t(num_gp_lanes, tau_e_t[i], alpha_e_t[i], od_to_edges_list_full, \
                                         demand_array_temp, VoT_array, el_indices, in_indices, \
                                         coeff_input, e, t)

            y_perturbed_values_at_e_t = \
                solve_DBCP_at_e_t(num_gp_lanes, tau_perturbed_e_t[i], alpha_perturbed_e_t[i], od_to_edges_list_full, \
                                                   demand_array_temp, VoT_array, el_indices, in_indices, \
                                                   coeff_input, e, t)
            
#             def welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
#                        demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                        coeff_input, e, t):

            welfare_at_e_t, obj_E_at_e_t, obj_R_at_e_t, obj_I_at_e_t = \
                welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                                       tau_e_t[i], alpha_e_t[i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                                       y = y_values_at_e_t, coeff_input = coeff_input, e = e, t = t)
            
            welfare_perturbed_at_e_t, obj_E_perturbed_at_e_t, obj_R_perturbed_at_e_t, obj_I_perturbed_at_e_t \
                = welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                              tau_perturbed_e_t[i], alpha_perturbed_e_t[i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                              y = y_perturbed_values_at_e_t, coeff_input = coeff_input, e = e, t = t)
            
            print()
            print("welfare_at_e_t:", welfare_at_e_t)
            print("obj_E_at_e_t:", obj_E_at_e_t)
            print("obj_R_at_e_t:", obj_R_at_e_t)
            print("obj_I_at_e_t:", obj_I_at_e_t)
            print()
            print("welfare_perturbed_at_e_t:", welfare_perturbed_at_e_t)
            print("obj_E_perturbed_at_e_t:", obj_E_perturbed_at_e_t)
            print("obj_R_perturbed_at_e_t:", obj_R_perturbed_at_e_t)
            print("obj_I_perturbed_at_e_t:", obj_I_perturbed_at_e_t)
            print()
            
            if welfare_at_e_t < welfare_min_array[e, t]:
                
                welfare_min_array[e, t] = welfare_at_e_t
                obj_E_min_at_e_t[e, t] = obj_E_at_e_t
                obj_R_min_at_e_t[e, t] = obj_R_at_e_t
                obj_I_min_at_e_t[e, t] = obj_I_at_e_t
                argmin_tau_new[e, t] = tau_e_t[i]
                argmin_alpha_new[e, t] = alpha_e_t[i]
                
                for key in list(y_values_at_e_t.keys()):
                    y_opt_DBCP_new[key] = y_values_at_e_t[key]
            
#             print("quantity:", tau[e, t, i] - eta[i] * (d/delta[i]) * w_i[0] \
#                                 * (welfare_perturbed_at_e_t - welfare_at_e_t))

            tau_e_t[i+1] = tau_e_t[i] - eta[i] * (d/delta[i]) * w_i[0] \
                                * (welfare_perturbed_at_e_t - welfare_at_e_t)

            alpha_e_t[i+1] = alpha_e_t[i] - eta[i] * (d/delta[i]) * w_i[1] \
                                * (welfare_perturbed_at_e_t - welfare_at_e_t)

            print("tau_e_t[:, :, i+1], before projection:\n", tau_e_t[i+1])
            print("alpha_e_t[:, :, i+1], before projection:\n", alpha_e_t[i+1])
            
            tau_e_t[i+1], alpha_e_t[i+1] \
                = proj_tau_alpha_at_e_t(T, num_edges, tau_e_t[i+1], alpha_e_t[i+1], \
                                        tau_max = tau_upper_limit)

            print("tau_e_t[:, :, i+1], after projection:\n", tau_e_t[i+1])
            print("alpha_e_t[:, :, i+1], after projection:\n", alpha_e_t[i+1])
        
            if i >= diffs_num_cols + 2:
                tau_diffs = np.max(np.absolute(tau_e_t[i - diffs_num_cols : i-1] - tau_e_t[i - diffs_num_cols+1 : i]))
                alpha_diffs = np.max(np.absolute(alpha_e_t[i-diffs_num_cols : i-1] - alpha_e_t[i-diffs_num_cols+1 : i]))

        #         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
        #         print("alpha[:, :, 0:10]:", alpha[:, :, 0:10])
                print("tau_diffs:\n", tau_diffs)
                print("alpha_diffs:\n", alpha_diffs)
                print()

                if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(alpha_diffs))) < error_bound:
                    print("Within error bound.")
                    break

            time_2_at_e_t = time.time()

            
            print()
            print("Iteration count:", i)
            print("Time:", time_2_at_e_t - time_1_at_e_t)
            print()

time_2 = time.time()
print("Time:", time_2 - time_1)


lambda_E: 1.0
lambda_R: 1.0
lambda_I: 1.0

e: 0
t: 0

Iter: 0
tau_e_t[i], before projection:
 0.92
alpha_e_t[i], before projection:
 0.8433825682471172
tau_perturbed_e_t[i], before projection:
 0.9201327758271736
alpha_perturbed_e_t[i], before projection:
 0.8430548937444192
tau_perturbed_e_t[i]: 0.9201327758271736
alpha_perturbed_e_t[i]: 0.8430548937444192

tau_feas.value: [0.92]

alpha_feas: 0.8430548937444192

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.92]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.843
tau[e, t, i], after projection:
 0.92
alpha[e, t, i], after projection:
 0.8433825682471172
tau_perturbed[e, t, i], after projection:
 0.92
alpha_perturbed[e, t, i], after projection:
 0.843

VoT_array.shape: (24, 5, 5)


prob.status: optimal

prob.status: optimal

welfare_at_e_t: 8569.967118757528
obj_E_at_e_t: 508.39494154849353
obj_R_at_e_t: 427.02134414937194
obj_I_at_e_t: 8488.593521358405

welfare_perturbed_at_e_t: 8569.832169400794
obj_E_perturbed_at_e_


prob.status: optimal

welfare_at_e_t: 7992.827962893007
obj_E_at_e_t: 488.4506333448345
obj_R_at_e_t: 521.0534637675945
obj_I_at_e_t: 8025.430793315767

welfare_perturbed_at_e_t: 7992.827962893007
obj_E_perturbed_at_e_t: 488.4506333448345
obj_R_perturbed_at_e_t: 521.0534637675945
obj_I_perturbed_at_e_t: 8025.430793315767

tau_e_t[:, :, i+1], before projection:
 0.58
alpha_e_t[:, :, i+1], before projection:
 0.667

tau_feas.value: [0.58]

alpha_feas: 0.667

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.58]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.667
tau_e_t[:, :, i+1], after projection:
 0.58
alpha_e_t[:, :, i+1], after projection:
 0.667

Iteration count: 6
Time: 0.3156280517578125


Iter: 7
tau_e_t[i], before projection:
 0.58
alpha_e_t[i], before projection:
 0.667
tau_perturbed_e_t[i], before projection:
 0.5801876753166917
alpha_perturbed_e_t[i], before projection:
 0.6670947214301456
tau_perturbed_e_t[i]: 0.5801876753166917
alpha_perturbed_e_t[i]: 0.66709


prob.status: optimal

welfare_at_e_t: 7929.72554296867
obj_E_at_e_t: 465.092479558318
obj_R_at_e_t: 172.15094540629076
obj_I_at_e_t: 7636.784008816643

welfare_perturbed_at_e_t: 7929.72554296867
obj_E_perturbed_at_e_t: 465.092479558318
obj_R_perturbed_at_e_t: 172.15094540629076
obj_I_perturbed_at_e_t: 7636.784008816643

tau_e_t[:, :, i+1], before projection:
 0.281
alpha_e_t[:, :, i+1], before projection:
 0.851

tau_feas.value: [0.281]

alpha_feas: 0.851

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.281]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.851
tau_e_t[:, :, i+1], after projection:
 0.281
alpha_e_t[:, :, i+1], after projection:
 0.851

Iteration count: 4
Time: 0.31679582595825195


Iter: 5
tau_e_t[i], before projection:
 0.281
alpha_e_t[i], before projection:
 0.851
tau_perturbed_e_t[i], before projection:
 0.28082165863456754
alpha_perturbed_e_t[i], before projection:
 0.8508613443341091
tau_perturbed_e_t[i]: 0.28082165863456754
alpha_perturbed_e_t[i]: 0


prob.status: optimal

welfare_at_e_t: 7811.445300960021
obj_E_at_e_t: 476.044573296228
obj_R_at_e_t: 526.0457832949213
obj_I_at_e_t: 7861.4465109587145

welfare_perturbed_at_e_t: 7811.445300960021
obj_E_perturbed_at_e_t: 476.044573296228
obj_R_perturbed_at_e_t: 526.0457832949213
obj_I_perturbed_at_e_t: 7861.4465109587145

tau_e_t[:, :, i+1], before projection:
 0.58
alpha_e_t[:, :, i+1], before projection:
 0.0

tau_feas.value: [0.58]

alpha_feas: 0.0

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.58]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.0
tau_e_t[:, :, i+1], after projection:
 0.58
alpha_e_t[:, :, i+1], after projection:
 0.0

Iteration count: 2
Time: 0.3134317398071289


Iter: 3
tau_e_t[i], before projection:
 0.58
alpha_e_t[i], before projection:
 0.0
tau_perturbed_e_t[i], before projection:
 0.5802422371630087
alpha_perturbed_e_t[i], before projection:
 -6.181550661009707e-05
tau_perturbed_e_t[i]: 0.5802422371630087
alpha_perturbed_e_t[i]: -6.1815506610


prob.status: optimal

welfare_at_e_t: 8046.107695865001
obj_E_at_e_t: 438.9511482912759
obj_R_at_e_t: 379.8651666977799
obj_I_at_e_t: 7987.021714271505

welfare_perturbed_at_e_t: 8046.1057398841585
obj_E_perturbed_at_e_t: 438.8998384537858
obj_R_perturbed_at_e_t: 379.820474005497
obj_I_perturbed_at_e_t: 7987.026375435869

tau_e_t[:, :, i+1], before projection:
 0.47745690918690303
alpha_e_t[:, :, i+1], before projection:
 0.8646192337466176

tau_feas.value: [0.477]

alpha_feas: 0.8646192337466176

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.477]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.865
tau_e_t[:, :, i+1], after projection:
 0.477
alpha_e_t[:, :, i+1], after projection:
 0.865

Iteration count: 0
Time: 0.3147101402282715


Iter: 1
tau_e_t[i], before projection:
 0.477
alpha_e_t[i], before projection:
 0.865
tau_perturbed_e_t[i], before projection:
 0.4769177753275465
alpha_perturbed_e_t[i], before projection:
 0.8652857051817665
tau_perturbed_e_t[i]: 0.476

...


prob.status: optimal

prob.status: optimal

welfare_at_e_t: 17911.76178547457
obj_E_at_e_t: 1163.4737515193983
obj_R_at_e_t: 0.0004460213625592171
obj_I_at_e_t: 16748.288479976534

welfare_perturbed_at_e_t: 17911.76178547457
obj_E_perturbed_at_e_t: 1163.4737515193983
obj_R_perturbed_at_e_t: 0.0004460213625592171
obj_I_perturbed_at_e_t: 16748.288479976534

tau_e_t[:, :, i+1], before projection:
 0.58
alpha_e_t[:, :, i+1], before projection:
 1.0

tau_feas.value: [0.58]

alpha_feas: 1.0

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.58]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 1.0
tau_e_t[:, :, i+1], after projection:
 0.58
alpha_e_t[:, :, i+1], after projection:
 1.0

Iteration count: 4
Time: 0.3163111209869385


Iter: 5
tau_e_t[i], before projection:
 0.58
alpha_e_t[i], before projection:
 1.0
tau_perturbed_e_t[i], before projection:
 0.5801952148731778
alpha_perturbed_e_t[i], before projection:
 1.0001136758092037
tau_perturbed_e_t[i]: 0.5801952148731778
alpha_p


prob.status: optimal

prob.status: optimal

welfare_at_e_t: 16876.609773854132
obj_E_at_e_t: 1273.2753550288553
obj_R_at_e_t: 0.0001701585479887609
obj_I_at_e_t: 15603.334588983824

welfare_perturbed_at_e_t: 16876.609773854132
obj_E_perturbed_at_e_t: 1273.2753550288553
obj_R_perturbed_at_e_t: 0.0001701585479887609
obj_I_perturbed_at_e_t: 15603.334588983824

tau_e_t[:, :, i+1], before projection:
 0.58
alpha_e_t[:, :, i+1], before projection:
 1.0

tau_feas.value: [0.58]

alpha_feas: 1.0

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.58]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 1.0
tau_e_t[:, :, i+1], after projection:
 0.58
alpha_e_t[:, :, i+1], after projection:
 1.0

Iteration count: 2
Time: 0.3142578601837158


Iter: 3
tau_e_t[i], before projection:
 0.58
alpha_e_t[i], before projection:
 1.0
tau_perturbed_e_t[i], before projection:
 0.580204976487658
alpha_perturbed_e_t[i], before projection:
 0.9998568754405863
tau_perturbed_e_t[i]: 0.580204976487658
alpha_p


prob.status: optimal

prob.status: optimal

welfare_at_e_t: 17843.187806994825
obj_E_at_e_t: 1129.017654145932
obj_R_at_e_t: 274.2413553213578
obj_I_at_e_t: 16988.41150817025

welfare_perturbed_at_e_t: 17843.178750597912
obj_E_perturbed_at_e_t: 1129.102814715225
obj_R_perturbed_at_e_t: 274.3482761728272
obj_I_perturbed_at_e_t: 16988.424212055514

tau_e_t[:, :, i+1], before projection:
 0.3540478309006162
alpha_e_t[:, :, i+1], before projection:
 0.8720302171430446

tau_feas.value: [0.354]

alpha_feas: 0.8720302171430446

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.354]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.872
tau_e_t[:, :, i+1], after projection:
 0.354
alpha_e_t[:, :, i+1], after projection:
 0.872

Iteration count: 0
Time: 0.31453394889831543


Iter: 1
tau_e_t[i], before projection:
 0.354
alpha_e_t[i], before projection:
 0.872
tau_perturbed_e_t[i], before projection:
 0.3542197238269019
alpha_perturbed_e_t[i], before projection:
 0.8722002742807748
ta


prob.status: optimal

prob.status: optimal

welfare_at_e_t: 17831.498571413984
obj_E_at_e_t: 1131.318709593846
obj_R_at_e_t: 283.94523666020837
obj_I_at_e_t: 16984.125098480345

welfare_perturbed_at_e_t: 17831.498571413984
obj_E_perturbed_at_e_t: 1131.318709593846
obj_R_perturbed_at_e_t: 283.94523666020837
obj_I_perturbed_at_e_t: 16984.125098480345

tau_e_t[:, :, i+1], before projection:
 0.354
alpha_e_t[:, :, i+1], before projection:
 0.872

tau_feas.value: [0.354]

alpha_feas: 0.872

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.354]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.872
tau_e_t[:, :, i+1], after projection:
 0.354
alpha_e_t[:, :, i+1], after projection:
 0.872

Iteration count: 6
Time: 0.3150210380554199


Iter: 7
tau_e_t[i], before projection:
 0.354
alpha_e_t[i], before projection:
 0.872
tau_perturbed_e_t[i], before projection:
 0.3542065904709611
alpha_perturbed_e_t[i], before projection:
 0.8720389172343856
tau_perturbed_e_t[i]: 0.354206590470961


prob.status: optimal

prob.status: optimal

welfare_at_e_t: 18143.361913451517
obj_E_at_e_t: 1222.9141624590698
obj_R_at_e_t: 3.483181932500171e-05
obj_I_at_e_t: 16920.447785824268

welfare_perturbed_at_e_t: 18143.361913451517
obj_E_perturbed_at_e_t: 1222.9141624590698
obj_R_perturbed_at_e_t: 3.483181932500171e-05
obj_I_perturbed_at_e_t: 16920.447785824268

tau_e_t[:, :, i+1], before projection:
 0.58
alpha_e_t[:, :, i+1], before projection:
 1.0

tau_feas.value: [0.58]

alpha_feas: 1.0

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.58]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 1.0
tau_e_t[:, :, i+1], after projection:
 0.58
alpha_e_t[:, :, i+1], after projection:
 1.0

Iteration count: 4
Time: 0.3150320053100586


Iter: 5
tau_e_t[i], before projection:
 0.58
alpha_e_t[i], before projection:
 1.0
tau_perturbed_e_t[i], before projection:
 0.5802238191508545
alpha_perturbed_e_t[i], before projection:
 0.9999694054903825
tau_perturbed_e_t[i]: 0.5802238191508545
alpha

In [44]:
obj_DBCP_init_min_as_array = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        obj_DBCP_init_min_as_array[e, t] = obj_DBCP_init_min[(e, t)]
        
obj_DBCP_init_min_as_array

array([[ 8575.58040367,  7926.16440034,  7932.7178661 ,  8039.91225185,
         8981.07215846],
       [13225.23898743, 11575.2271517 , 12470.13067853, 12158.0711323 ,
        13065.19555636],
       [40731.94825647, 39685.63585097, 39260.70602525, 40069.696903  ,
        42219.67352978],
       [ 9639.81214758,  9638.97268494, 10609.85577855,  9713.67221275,
        13893.78587682],
       [57579.2170853 , 57294.94681737, 55017.80386818, 56756.61259347,
        58296.63057013],
       [11526.73862484, 11255.5199278 , 10814.352635  , 11026.0343199 ,
        11222.89657182],
       [19217.48244334, 18061.95974275, 17124.00977712, 17843.9488788 ,
        18600.44543796]])

In [45]:
np.all(welfare_min_array <= obj_DBCP_init_min_as_array)

True

In [64]:
print("argmin_tau:\n", argmin_tau)
print()
print("argmin_tau_new:\n", argmin_tau_new)
print()
print("init_alpha:\n", init_alpha)
print()
print("argmin_alpha_new:", argmin_alpha_new)
print()
print("welfare_min_array:", welfare_min_array)
print()
print("welfare_min_array - obj_DBCP_init_min_as_array:", \
      welfare_min_array - obj_DBCP_init_min_as_array)
print()
print("np.sum(welfare_min_array - obj_DBCP_init_min_as_array):", \
      np.sum(welfare_min_array - obj_DBCP_init_min_as_array))
print()
print("np.sum(welfare_min_array) - obj_CBCP_opt[total]:", \
      np.sum(welfare_min_array) - obj_CBCP_opt['total'])



argmin_tau:
 [[0.92 0.28 0.46 0.48 0.93]
 [0.95 0.32 0.79 0.58 0.71]
 [0.52 2.24 2.07 1.35 2.2 ]
 [1.26 0.86 1.71 1.38 2.46]
 [2.13 2.29 2.26 0.59 1.31]
 [0.53 0.44 0.69 0.51 0.38]
 [0.78 0.26 0.48 0.36 0.74]]

argmin_tau_new:
 [[0.58  0.28  0.58  0.48  0.58 ]
 [0.58  0.32  0.    0.58  0.   ]
 [0.52  0.    0.58  1.35  0.58 ]
 [1.26  0.    0.    0.58  0.58 ]
 [2.13  0.58  0.    0.59  0.32 ]
 [0.53  0.44  0.58  0.415 0.58 ]
 [0.58  0.58  0.58  0.354 0.58 ]]

init_alpha:
 [[0.84338257 0.85084862 0.85706593 0.86353093 0.84168641]
 [0.83200002 0.84995156 0.83773352 0.84321766 0.83352961]
 [0.85407664 0.86165158 0.85706593 0.87755727 0.85395944]
 [0.85393105 0.92504674 0.85680169 0.87571159 0.8467413 ]
 [0.86209681 0.92352439 0.9169529  0.87957208 0.9254939 ]
 [0.84338257 0.92125546 0.84409076 0.87397343 0.92228554]
 [0.86032559 0.91955794 0.91636279 0.88337078 0.86165457]]

argmin_alpha_new: [[0.667      0.85084862 0.         0.86353093 0.812     ]
 [1.         0.84995156 1.         1.     

In [53]:
y_opt_DBCP_new

{(3, 3, 0, 0, 0): 1.9481879815823544e-06,
 (3, 3, 0, 1, 0): 56.292808366212014,
 (1, 3, 0, 0, 0): 2.417504879931749e-06,
 (1, 3, 0, 1, 0): 21.56767995169512,
 (4, 3, 0, 0, 0): 1.976656387413785e-06,
 (4, 3, 0, 1, 0): 213.41396855706358,
 (5, 3, 0, 0, 0): 2.4784178493770357e-06,
 (5, 3, 0, 1, 0): 53.35323886298215,
 (2, 3, 0, 0, 0): 2.0069913488312637e-06,
 (2, 3, 0, 1, 0): 250.36177308280867,
 (0, 3, 0, 0, 0): 2.2423883513776896e-06,
 (0, 3, 0, 1, 0): 134.75871347561167,
 (6, 3, 0, 0, 0): 2.6711275582690142e-06,
 (6, 3, 0, 1, 0): 336.83460110047247,
 (4, 4, 0, 0, 0): 2.8759496814508953e-05,
 (4, 4, 0, 1, 0): 454.95975387829714,
 (5, 4, 0, 0, 0): 6.097140389813845e-05,
 (5, 4, 0, 1, 0): 113.73934897912609,
 (0, 4, 0, 0, 0): 0.000367701866321626,
 (0, 4, 0, 1, 0): 287.28071262423373,
 (2, 4, 0, 0, 0): 14.313110117040665,
 (2, 4, 0, 1, 0): 519.4126740516693,
 (3, 4, 0, 0, 0): 120.00600748872084,
 (3, 4, 0, 1, 0): 2.9045159138263443e-05,
 (1, 4, 0, 0, 0): 45.97834595288845,
 (1, 4, 0, 1, 0

# Storing data

In [54]:
# Change argmin_y format from dict to array:

# y_opt_DBCP_new


y_opt_DBCP_new_array = np.zeros((len(list(y_opt_DBCP_new.keys())) , 6))

for row_index, key in enumerate(list(y_opt_DBCP_new.keys())):
#     print(key)
    
    y_opt_DBCP_new_array[row_index, :5] = np.array(key)
    
    if y_opt_DBCP_new[key] <= 0.0 and y_opt_DBCP_new_array[row_index, -1] >= -1E-3:
        y_opt_DBCP_new_array[row_index, -1] = 0.0
    else:
        y_opt_DBCP_new_array[row_index, -1] = y_opt_DBCP_new[key]
#     print("row_index, key", (row_index, key))



In [55]:
directory_path = "../data/opt_DBCP_values___" + str(num_el) + "_el_groups/"

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

filename = filename_segment + "___y_DBCP.csv"


column_names = ["od", "g", "e", "k", "t", "flows (y)"]

df_y_opt_DBCP_new = pd.DataFrame(y_opt_DBCP_new_array, columns = column_names)
df_y_opt_DBCP_new["od"] = df_y_opt_DBCP_new["od"].astype(int)
df_y_opt_DBCP_new["g"] = df_y_opt_DBCP_new["g"].astype(int)
df_y_opt_DBCP_new["e"] = df_y_opt_DBCP_new["e"].astype(int)
df_y_opt_DBCP_new["k"] = df_y_opt_DBCP_new["k"].astype(int)
df_y_opt_DBCP_new["t"] = df_y_opt_DBCP_new["t"].astype(int)

df_y_opt_DBCP_new.to_csv(directory_path + filename, index=False)

df_y_opt_DBCP_new

# y_opt_DBCP_new


,od,g,e,k,t,flows (y)
0,3,3,0,0,0,1.948188e-06
1,3,3,0,1,0,5.629281e+01
2,1,3,0,0,0,2.417505e-06
3,1,3,0,1,0,2.156768e+01
4,4,3,0,0,0,1.976656e-06
...,...,...,...,...,...,...
3995,23,2,6,1,4,1.922200e+02
3996,11,2,6,0,4,8.475307e+01
3997,11,2,6,1,4,8.526513e-14
3998,6,2,6,0,4,2.342532e+02


In [56]:
y_agg_DBCP_new, x_DBCP_new = \
    compute_y_agg_x(y = y_opt_DBCP_new, \
                    edge_to_od_dict = edge_to_od_dict, \
                    num_edges = num_edges, \
                    T = T)

travel_times_DBCP_new, avg_travel_times_DBCP_new = \
    compute_travel_times(y_agg = y_agg_DBCP_new, \
                         x = x_DBCP_new, \
                         edge_to_od_dict = edge_to_od_dict, \
                         coeff_input = coeff_input, \
                         num_gp_lanes = num_gp_lanes, \
                         num_edges = num_edges, \
                         T = T)

percent_on_express_DBCP_new = \
    compute_percent_on_express(y_agg = y_agg_DBCP_new, \
                               x = x_DBCP_new, \
                               num_edges = num_edges, \
                               T = T)
    
    
    

In [57]:
avg_travel_times_DBCP_new

{(0, 'ex'): 1.4042260093819592,
 (0, 'gp'): 1.6658055036153407,
 (0, 'el'): 1.649630361890256,
 (0, 'in'): 1.5992525664595332,
 (1, 'ex'): 2.360440111823434,
 (1, 'gp'): 2.3916121967209563,
 (1, 'el'): 2.3833963225807873,
 (1, 'in'): 2.3858671557569555,
 (2, 'ex'): 5.979540409259738,
 (2, 'gp'): 6.2336701401461205,
 (2, 'el'): 6.186045152850596,
 (2, 'in'): 6.175398163651158,
 (3, 'ex'): 1.4785443448699485,
 (3, 'gp'): 1.726469050842028,
 (3, 'el'): 1.6902043172863959,
 (3, 'in'): 1.662559951166153,
 (4, 'ex'): 7.385598184241493,
 (4, 'gp'): 7.652083524263939,
 (4, 'el'): 7.570070945456442,
 (4, 'in'): 7.608555949275288,
 (5, 'ex'): 1.5547912734290692,
 (5, 'gp'): 1.8195146982674444,
 (5, 'el'): 1.7903694031786526,
 (5, 'in'): 1.774659007491618,
 (6, 'ex'): 2.609890721722517,
 (6, 'gp'): 2.7061701264912132,
 (6, 'el'): 2.6964981181573893,
 (6, 'in'): 2.6872590547148323,
 'ex': 22.77303105472816,
 'gp': 24.19532524034705,
 'el': 3.6978408909730804,
 'in': 3.6565501108661502}

In [58]:
opt_data_array = np.zeros((num_edges, 20))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau_new

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau_new, axis=1)

# argmin_alpha
opt_data_array[:, 6:11] = argmin_alpha_new

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 11] = np.array([percent_on_express_DBCP_new[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 12] = np.array([percent_on_express_DBCP_new[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 13] = np.array([percent_on_express_DBCP_new[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 14] = np.array([avg_travel_times_DBCP_new[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 15] = np.array([avg_travel_times_DBCP_new[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 16] = np.array([np.sum(obj_E_min_at_e_t[e, :]) for e in range(num_edges)])
opt_data_array[:, 17] = np.array([np.sum(obj_I_min_at_e_t[e, :]) for e in range(num_edges)])
opt_data_array[:, 18] = np.array([np.sum(obj_R_min_at_e_t[e, :]) for e in range(num_edges)])
opt_data_array[:, 19] = np.array([np.sum(welfare_min_array[e, :]) for e in range(num_edges)])

opt_data_array = np.round(opt_data_array, decimals=2)

In [59]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)"]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]
# column_names += ["alpha (time-averaged)"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_DBCP_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_DBCP_save

,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),alpha (t=1),alpha (t=2),alpha (t=3),alpha (t=4),alpha (t=5),% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.58,0.28,0.58,0.48,0.58,0.50,0.67,0.85,0.0,0.86,0.81,19.83,9.84,24.34,1.40,1.67,2365.92,39835.44,2150.45,40050.90
e=2,0.58,0.32,0.00,0.58,0.00,0.30,1.00,0.85,1.0,1.00,0.00,24.23,52.77,9.43,2.36,2.39,4030.35,55906.13,223.34,59713.14
e=3,0.52,0.00,0.58,1.35,0.58,0.61,0.85,1.00,0.0,0.88,1.00,23.21,30.66,19.40,5.98,6.23,13611.49,189965.50,2599.30,200977.69
e=4,1.26,0.00,0.00,0.58,0.58,0.48,0.85,1.00,0.0,0.82,0.84,23.46,16.20,27.02,1.48,1.73,3215.57,46494.46,2615.91,47094.13
e=5,2.13,0.58,0.00,0.59,0.32,0.72,0.86,1.00,1.0,0.88,1.00,23.83,48.05,11.80,7.39,7.65,19057.24,265394.27,2651.55,281799.96
e=6,0.53,0.44,0.58,0.42,0.58,0.51,0.84,0.92,0.0,0.77,0.00,15.26,12.99,16.36,1.55,1.82,3792.08,53230.79,1956.04,55066.83
e=7,0.58,0.58,0.58,0.35,0.58,0.53,0.00,1.00,1.0,0.87,1.00,21.85,51.19,7.45,2.61,2.71,6124.33,84250.50,898.53,89476.29


In [60]:
directory_to_save = "../data/opt_DBCP_values___" + str(num_el) + "_el_groups/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

filename = filename_segment + '___tau_alpha_stats_DBCP.csv'

df_opt_DBCP_save.to_csv(directory_to_save + filename)

# <font color='red'>STOP, END here.</font>

## <font color='red'>Continue in Bilevel_Opt_aux_compare_CBCP_DBCP.</font> 

In [ ]:
# # y_agg_el_at_e_t
# print("y_agg_el_ex:", y_agg_el_ex)

# e_temp = 1
# sum([demand_array[od, g] for od in edge_to_od_dict[e_temp] for g in el_indices])

In [ ]:
# y_DBCP_init_at_e_t

# assert y_DBCP_init_at_e_t == extract_y_at_e_t(y_DBCP_init, 6, 4)

In [ ]:
# for e in range(num_edges):
#     print("Edge:", e)
    
#     assert equals(obj_DBCP_init_together[(e, 'E')], obj_DBCP_init[(e, 'E')])
#     assert equals(obj_DBCP_init_together[(e, 'R')], obj_DBCP_init[(e, 'R')])
#     assert equals(obj_DBCP_init_together[(e, 'I')], obj_DBCP_init[(e, 'I')])
    

# Old Code

In [ ]:
# list_1 = [1, 2, 3]
# list_2 = [2, 3, 4]

# [x for x in list_1 if x not in list_2]


In [ ]:
edge_to_od_dict

In [ ]:
# init_DBCP_key_list_comp = [(od, g, e, k, t) for e in range(num_edges) for t in range(T) \
#                               for k in [0, 1] for g in (el_indices + in_indices) for od in edge_to_od_dict[e]]

# list(init_DBCP_flows.keys()) == init_DBCP_key_list_comp

# keys_missing_init_DBCP_flows = [key for key in list(init_DBCP_flows.keys()) \
#                                 if key not in init_DBCP_key_list_comp]
# keys_extra_init_DBCP_flows = [key for key in init_DBCP_key_list_comp \
#                                 if key not in list(init_DBCP_flows.keys())]

# print("keys_missing_init_DBCP_flows:", keys_missing_init_DBCP_flows)
# print("\n")
# print("keys_extra_init_DBCP_flows:", keys_extra_init_DBCP_flows)

In [ ]:
y_in_el_total_init_DBCP = {}
x_init_DBCP = {}

for e in range(num_edges):
    for t in range(T):
        for k in [0, 1]:
            y_in_el_total_init_DBCP[(e, k, t, "in")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in in_indices)
            y_in_el_total_init_DBCP[(e, k, t, "el")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in el_indices)
            
            x_init_DBCP[(e, k, t)] = y_in_el_total_init_DBCP[(e, k, t, "in")] \
                                        + y_in_el_total_init_DBCP[(e, k, t, "el")]


In [ ]:
y_init_DBCP

In [ ]:
y_in_el_total_init_DBCP

In [ ]:
x_init_DBCP

In [ ]:
# lambda_R = 1

travel_times_DBCP = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times_DBCP[(e, 0, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times_DBCP[(e, 1, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

# INCOMPLETE:
        
edge_demand_DBCP = {}
avg_travel_time_DBCP = {}
percent_on_express_DBCP = {}
obj_E_DBCP = np.zeros(num_edges)
obj_I_DBCP = np.zeros(num_edges)
obj_R_DBCP = np.zeros(num_edges)
obj_DBCP = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time_DBCP[e, 'el'] = 0.0
    avg_travel_time_DBCP[e, 'in'] = 0.0
    avg_travel_time_DBCP[e, 'ex'] = 0.0
    avg_travel_time_DBCP[e, 'gp'] = 0.0
    
    percent_on_express_DBCP[e, 'el'] = 0.0
    percent_on_express_DBCP[e, 'in'] = 0.0
    percent_on_express_DBCP[e, 'all'] = 0.0
    
    obj_E_DBCP[e] = 0.0
    obj_I_DBCP[e] = 0.0
    obj_R_DBCP[e] = 0.0
    obj_DBCP[e] = 0.0
    
    for t in range(T):
        edge_demand_DBCP[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand_DBCP[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))
    percent_on_express_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))
    percent_on_express_DBCP[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] + edge_demand_DBCP[e, t, 'in'] for t in range(T))
    
    avg_travel_time_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))    
    avg_travel_time_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))    
#     avg_travel_time_DBCP[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time_DBCP[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time_DBCP[e, 'ex'] += sum(travel_times_DBCP[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time_DBCP[e, 'gp'] += sum(travel_times_DBCP[(e, 1, t)] for t in range(T)) / T 
    
    obj_E_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t]) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
    
#                     + sum( y_init_DBCP[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \

    obj_I_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t])

    obj_DBCP[e] = lambda_E * obj_E_DBCP[e] - lambda_R * obj_R_DBCP[e] + lambda_I * obj_I_DBCP[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time_DBCP
# percent_on_express_DBCP
# obj_R_DBCP

sum(obj_DBCP)

In [ ]:
# column_names = []
# column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
# column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

# row_names = ["e=" + str(k+1) for k in range(num_edges) ]

# df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# # df_inits_save

# # random_string = ""
# # for idx_rand_str in range(10):
# #     random_string += str(np.random.randint(1, 9))

# directory_to_save = "../data/old___opt_tolls_subsidies_metrics/"
# random_filename = "inits___" + random_string + '.csv'

# df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
print("first(welfares):", welfares[0])
print("min(welfares):", min(welfares))
print("max(welfares):", max(welfares))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
